# Aggregations

Aggregations are an important step while processing dataframes and tabular data
in general. And therefore, they should be as simple as possible to implement.
Some notable data aggregation semantics are provided by pandas, spark and the SQL
language.

When designing an aggregation API method, the following characteristics make in
my opinion a good aggregation method.

-   easily perform aggregation on a column or a set of columns
-   easily perform multiple aggregation functions on the same columns
-   selectively perform differently aggregations on different columns

As an nice to have to this list, it would be nice to apply aggregation functions
by passing the function name as a string. A good aggregation method should allow
all the above with minimal amount of code required.

## Getting started

Let's start spark using datafaucet.

In [1]:
import datafaucet as dfc

In [2]:
# let's start the engine
dfc.engine('spark')

## Generating Data

In [3]:
df = dfc.range(1000)

In [10]:
df = (df
    .cols.create('g').randchoice([2,7])
    .cols.create('n').randchoice(['Stacy', 'Sandra'])
    .cols.create('x').randint(0,100)
    .cols.create('y').randint(0,100)
)

In [11]:
df.data.grid(5)

,id,g,n,x,y
0,0,2,Stacy,99,22
1,1,2,Sandra,10,40
2,2,2,Sandra,39,42
3,3,2,Stacy,92,86
4,4,2,Stacy,21,79


## Pandas
Let's start by looking how Pandas does aggregations. Pandas is quite flexible on the points noted above and uses hierachical indexes on both columns and rows to store the aggregation names and the groupby values. Here below a simple aggregation and a more complex one with groupby and multiple aggregation functions.

In [12]:
pf = df.data.collect()

In [13]:
pf[['n', 'x', 'y']].agg(['max'])

,n,x,y
max,Stacy,99,99


In [14]:
agg = (pf[['g','n', 'x', 'y']]
           .groupby(['g', 'n'])
           .agg({
               'n': 'count',
               'x': ['min', max],
               'y':['min', 'max']
           }))
agg

n   x       y    
         count min max min max
g n                           
2 Sandra   243   0  99   0  99
  Stacy    264   0  99   0  99
7 Sandra   290   0  99   0  99
  Stacy    203   0  99   1  99

### Stacking 
In pandas, you can stack the multiple column index and move it to a column, as below. The choice of stacking or not after aggregation depends on wht you want to do later with the data. Next to the extra index, stacking also explicitely code NaN / Nulls for evry aggregation which is not shared by each column (in case of dict of aggregation functions.

In [15]:
agg = pf[['g', 'n', 'x', 'y']].groupby(['g', 'n']).agg(['min', 'max', 'mean'])
agg = agg.stack(0)
agg

max       mean  min
g n                            
2 Sandra x   99  45.946502    0
         y   99  48.201646    0
  Stacy  x   99  51.753788    0
         y   99  49.443182    0
7 Sandra x   99  48.289655    0
         y   99  48.541379    0
  Stacy  x   99  51.108374    0
         y   99  49.822660    1

### Index as columns
Index in pandas is not the same as column data, but you can easily move from one to the other, as shown below, by combine the name information of the various index levels with the values of each level.

In [16]:
agg.index.names

FrozenList(['g', 'n', None])

In [17]:
# for example these are the value from the first level of the index
agg.index.get_level_values(0)

Int64Index([2, 2, 2, 2, 7, 7, 7, 7], dtype='int64', name='g')

The following script will iterate through all the levels and create a column with the name of the original index level otherwise will use `_<level#>` if no name is available. Remember that pandas allows indexes to be nameless.

In [18]:
levels = agg.index.names
for (name, lvl) in zip(levels, range(len(levels))):
    agg[name or f'_{lvl}'] = agg.index.get_level_values(lvl)

In [19]:
#now the index is standard columns, drop the index
agg.reset_index(inplace=True, drop=True)
agg

,max,mean,min,g,n,_2
0,99,45.946502,0,2,Sandra,x
1,99,48.201646,0,2,Sandra,y
2,99,51.753788,0,2,Stacy,x
3,99,49.443182,0,2,Stacy,y
4,99,48.289655,0,7,Sandra,x
5,99,48.541379,0,7,Sandra,y
6,99,51.108374,0,7,Stacy,x
7,99,49.822660,1,7,Stacy,y


## Spark (Python)
Spark aggregation is a bit simpler, but definitely very flexible, so we can achieve the same result with a little more work in some cases. Here below a simple example and a more complex one, reproducing the same three cases as above.

In [20]:
df.select('n', 'x', 'y').agg({'n':'max', 'x':'max', 'y':'max'}).toPandas()

,max(x),max(y),max(n)
0,99,99,Stacy


Or with a little more work we can exactly reproduce the pandas case:

In [21]:
from pyspark.sql import functions as F

df.select('n', 'x', 'y').agg(
    F.lit('max').alias('_idx'),
    F.max('n').alias('n'), 
    F.max('x').alias('x'), 
    F.max('y').alias('y')).toPandas()

,_idx,n,x,y
0,max,Stacy,99,99


More complicated aggregation cannot be called by string and must be provided by functions. Here below a way to reproduce groupby aggregation as in the second pandas example:

In [22]:
(df
    .select('g', 'n', 'x', 'y')
    .groupby('g', 'n')
    .agg(
        F.count('n').alias('n_count'),
        F.min('x').alias('x_min'),
        F.max('x').alias('x_max'),
        F.min('y').alias('y_min'),
        F.max('y').alias('y_max')
    )
).toPandas()
        

,g,n,n_count,x_min,x_max,y_min,y_max
0,2,Sandra,277,0,99,0,99
1,2,Stacy,226,0,99,0,99
2,7,Sandra,232,0,99,0,99
3,7,Stacy,265,0,99,0,99


### Stacking

Stacking, as in pandas, can be used to expose the column name on a different index column, unfortunatel stack is currently available only in the SQL initerface and not very flexible as in the pandas counterpart (https://spark.apache.org/docs/2.3.0/api/sql/#stack)

You could use pyspark `expr` to call the SQL function as explained here (https://stackoverflow.com/questions/42465568/unpivot-in-spark-sql-pyspark). However, another way would be to union the various results as shown here below.

In [23]:
from pyspark.sql import functions as F

(df
    .select('g', 'x')
    .groupby('g')
    .agg(
        F.lit('x').alias('_idx'),
        F.min('x').alias('min'),
        F.max('x').alias('max'),
        F.mean('x').alias('mean')
    )
).union(
df
    .select('g', 'y')
    .groupby('g')
    .agg(
        F.lit('y').alias('_idx'),
        F.min('y').alias('min'),
        F.max('y').alias('max'),
        F.mean('y').alias('mean')
    )
).toPandas()

,g,_idx,min,max,mean
0,7,x,0,99,50.058824
1,2,x,0,99,48.118451
2,7,y,0,99,49.313278
3,2,y,0,99,48.625483


### Generating aggregation code

The code above looks complicated, but is very regular, hence we can generate it! What we need is a to a list of lists for the aggregation functions as shown here below:

In [24]:
dfs = []
for c in ['x','y']:
    print(' '*2, f'col: {c}')
    aggs = []
    for func in [F.min, F.max, F.mean]:
        f = func(c).alias(func.__name__)
        aggs.append(f)
        print(' '*4, f'func: {f}')
        
    dfs.append(df.select('g', c).groupby('g').agg(*aggs))

   col: x
     func: Column<b'min(x) AS `min`'>
     func: Column<b'max(x) AS `max`'>
     func: Column<b'avg(x) AS `mean`'>
   col: y
     func: Column<b'min(y) AS `min`'>
     func: Column<b'max(y) AS `max`'>
     func: Column<b'avg(y) AS `mean`'>


The dataframes in this generator have all the same columns and can be reduced with union calls

In [25]:
from functools import reduce

reduce(lambda a,b: a.union(b), dfs).toPandas()

,g,min,max,mean
0,7,0,99,47.823308
1,2,0,99,50.779915
2,7,0,99,49.661504
3,2,0,99,48.375912


## Meet datafaucet agg

One of the goal of datafaucet is to simplify analytics, data wrangling and data
discovery over a set of engine with an intuitive interface. So the sketched
solution above is available, with a few extras. See below the examples

The code here below attempt to produce readable code, engine agnostic data
aggregations. The aggregation api is always in the form:   

`df.cols.get(...).groupby(...).agg(...)`

Alternativaly, you can `find` instead of `get`

In [26]:
# simple aggregation by name
d = df.cols.get('x').agg('distinct')
d.data.grid()

,x
0,100


In [27]:
# simple aggregation (multiple) by name
d = df.cols.get('x').agg(['distinct', 'avg'])
d.data.grid()

,x_distinct,x_avg
0,100,49.207


In [28]:
# simple aggregation (multiple) by name (stacked)
d = df.cols.get('x').agg(['distinct', 'avg'], stack=True)
d.data.grid()

,_idx,distinct,avg
0,x,100,49.207


In [29]:
# simple aggregation (multiple) by name (stacked, custom index name)
d = df.cols.get('x').agg(['distinct', 'avg'], stack='colname')
d.data.grid()

,colname,distinct,avg
0,x,100,49.207


In [30]:
# simple aggregation (multiple) by name and function
d = df.cols.get('x').agg(['distinct', F.min, F.max, 'avg'])
d.data.grid()

,x_distinct,x_min,x_max,x_avg
0,100,0,99,49.207


In [31]:
# multiple aggregation by name and function
d = df.cols.get('x', 'y').agg(['distinct', F.min, F.max, 'avg'])
d.data.grid()

,x_distinct,x_min,x_max,x_avg,y_distinct,y_min,y_max,y_avg
0,100,0,99,49.207,100,0,99,48.957


In [32]:
# multiple aggregation (multiple) by name and function
d = df.cols.get('x', 'y').agg({
    'x':['distinct', F.min], 
    'y':['distinct', 'max']})

d.data.grid()

,x_distinct,x_min,x_max,y_distinct,y_min,y_max
0,100,0,None,100,None,99


In [33]:
# multiple aggregation (multiple) by name and function (stacked)
d = df.cols.get('x', 'y').agg({
    'x':['distinct', F.min], 
    'y':['distinct', 'max']}, stack=True)
d.data.grid()

,_idx,distinct,min,max
0,x,100,0.0,NaN
1,y,100,NaN,99.0


In [34]:
# grouped by, multiple aggregation (multiple) by name and function (stacked)
d = df.cols.get('x', 'y').groupby('g','n').agg({
    'x':['distinct', F.min], 
    'y':['distinct', 'max']}, stack=True)
d.data.grid()

,g,n,_idx,distinct,min,max
0,2,Sandra,x,92,0.0,NaN
1,2,Stacy,x,89,0.0,NaN
2,7,Sandra,x,88,0.0,NaN
3,7,Stacy,x,95,0.0,NaN
4,2,Sandra,y,92,NaN,99.0
5,2,Stacy,y,87,NaN,99.0
6,7,Sandra,y,90,NaN,99.0
7,7,Stacy,y,98,NaN,99.0


### Extended list of aggregation

An extended list of aggregation is available, both by name and by function in the datafaucet library

In [35]:
from datafaucet.spark import functions as FF

In [36]:
d = df.cols.get('x', 'y').groupby('g','n').agg([
        'typeof',
        ('uniq', FF.distinct),
        'one',
        'percentiles',
    ], stack=True)

d.data.grid()

,g,n,_idx,typeof,uniq,one,percentiles
0,7,Sandra,x,int,94,17,"{25: 20, 10: 7, 90: 89, 75: 71}"
1,2,Sandra,x,int,91,3,"{25: 23, 10: 9, 90: 90, 75: 77}"
2,7,Stacy,x,int,91,30,"{25: 26, 10: 10, 90: 91, 75: 77}"
3,2,Stacy,x,int,85,95,"{25: 27, 10: 11, 90: 87, 75: 75}"
4,2,Stacy,y,int,90,36,"{25: 24, 10: 7, 90: 91, 75: 73}"
5,7,Sandra,y,int,93,17,"{25: 22, 10: 10, 90: 91, 75: 77}"
6,2,Sandra,y,int,94,3,"{25: 29, 10: 10, 90: 89, 75: 76}"
7,7,Stacy,y,int,88,30,"{25: 21, 10: 9, 90: 90, 75: 70}"


In [37]:
from datafaucet.spark import functions as FF
df.cols.get('x', 'y').groupby('g','n').agg(FF.topn(n=3, others=-1)).data.grid()

,g,n,x,y
0,2,Sandra,"{-1: 267, 90: 5, 68: 5, 95: 5}","{48: 6, 56: 5, -1: 236, 39: 7}"
1,2,Stacy,"{-1: 239, 82: 6, 36: 6, 46: 6}","{64: 6, -1: 279, 29: 9, 94: 6}"
2,7,Sandra,"{-1: 247, 36: 7, 69: 8, 14: 9}","{-1: 224, 10: 7, 42: 7, 11: 7}"
3,7,Stacy,"{24: 6, -1: 167, 57: 6, 22: 6}","{-1: 182, 99: 7, 14: 6, 31: 6}"


In [38]:
from datafaucet.spark import functions as FF
df.cols.get('x', 'y').groupby('g','n').agg(FF.topn_count(n=3)).data.grid()

,g,n,x,y
0,2,Sandra,18,18
1,2,Stacy,22,23
2,7,Sandra,20,21
3,7,Stacy,22,20


In [41]:
from datafaucet.spark import functions as FF
test_all = [(x[0], x[1]) for x in FF.all.items()]
list(FF.all.keys())

['typeof',
 'integer',
 'boolean',
 'top3',
 'top3_count',
 'top3_values',
 'percentiles',
 'null',
 'zero',
 'empty',
 'pos',
 'neg',
 'distinct',
 'sum',
 'count',
 'first',
 'one',
 'min',
 'max',
 'avg',
 'stddev',
 'skewness',
 'kurtosis',
 'sum_pos',
 'sum_neg',
 'digits_only',
 'spaces_only']

In [40]:
res = df.cols.get('x', 'y').groupby('g','n').agg(test_all, stack=True)
res.data.grid(axis=1)

,0,1,2,3,4,5,6,7
g,7,2,7,2,2,7,2,7
n,Sandra,Sandra,Stacy,Stacy,Stacy,Sandra,Sandra,Stacy
_idx,x,x,x,x,y,y,y,y
typeof,int,int,int,int,int,int,int,int
integer,195,267,257,281,292,218,224,266
boolean,2,8,2,11,9,7,7,3
top3,"{4: 7, 69: 7, 2: 7}","{9: 6, 87: 6, 67: 6}","{65: 8, 30: 7, 71: 9}","{77: 7, 22: 7, 43: 7}","{29: 10, 69: 8, 37: 7}","{17: 5, 10: 11, 79: 6}","{88: 6, 86: 5, 43: 5}","{81: 6, 42: 7, 99: 7}"
top3_count,21,18,24,21,25,22,16,20
top3_values,"[69, 2, 4]","[87, 9, 67]","[71, 65, 30]","[77, 22, 43]","[29, 69, 37]","[10, 79, 17]","[88, 86, 43]","[99, 42, 81]"
percentiles,"{25: 25, 10: 6, 90: 91, 75: 76}","{25: 21, 10: 9, 90: 87, 75: 72}","{25: 26, 10: 10, 90: 91, 75: 79}","{25: 22, 10: 9, 90: 87, 75: 72}","{25: 24, 10: 8, 90: 90, 75: 74}","{25: 25, 10: 10, 90: 90, 75: 72}","{25: 21, 10: 8, 90: 89, 75: 73}","{25: 26, 10: 11, 90: 90, 75: 75}"
